In [36]:
from pathlib import Path
import numpy as np
import re
from math import prod
from collections import defaultdict, deque
from copy import copy

In [41]:
data = Path('../Data/Day17.txt').read_text().splitlines()

In [42]:
[a, b, c] = [int(data[i].split(':')[1].strip()) for i in range(3)]
program = [int(num) for num in data[-1].split(': ')[1].split(',')]

In [125]:
def execute_program(a, b, c, program):
    combo = {
        0: 0,
        1: 1,
        2: 2,
        3: 3,
        4: a,
        5: b,
        6: c,
    }

    outputs = []

    pointer = 0

    while pointer < len(program):
        opcode = program[pointer]
        operand = program[pointer+1]

        match opcode:
            case 0:
                a = int(a / 2**combo[operand])
                combo[4] = a
            case 1:
                b = b ^ operand
                combo[5] = b
            case 2:
                b = combo[operand] % 8
                combo[5] = b
            case 3:
                if a != 0:
                    pointer = operand
                    continue
            case 4:
                b = b ^ c
                combo[5] = b
            case 5:
                outputs.append(combo[operand] % 8)
            case 6:
                b = int(a / 2 ** combo[operand])
                combo[5] = b
            case 7:
                c = int(a / 2**combo[operand])
                combo[6] = c
        
        pointer += 2

    return outputs

In [97]:
program, len(program)

([2, 4, 1, 2, 7, 5, 1, 3, 4, 4, 5, 5, 0, 3, 3, 0], 16)

In [ ]:
# (2, 4) Set B to A % 8
# (1, 2) Set B to B ^ 2 
# (7, 5) Set C to A // 2**B
# (1, 3) Set B to B ^ 3
# (4, 4) Set B to B ^ C
# (5, 5) Output B % 8
# (0, 3) Set A = A // 2**3
# (3, 0) If A != 0, jump to start

# Recognize that program is 16 numbers
# That requires 8**15 <= A < 8**16, aka A is a 48 bit number

# The state of B, C do not matter at the start of the cycle 
# as B is immediately determined by A
# and C is determined by A and B
# Thus the output is only determined by value of A at beginning of cycle
# And the next output is simply determined by A // 8

# B % 8 only cares about the last 3 bits of B
# So we only care about last 3 bits of B ^ C
# So we only care about last 3 bits of B and C
# B is based off of last 3 bits of A
# C is based off of last 3 bits after right shifting anywhere from 0 to 7 bits based on B
# Therefore B tells us last three bits of A at that cycle
# C tells us bits B to B+3
# Only 64 combinations of B and C to consider

# Some combinations of B and C cannot happen
# E.G. B,C cannot start as 0 and 5 as that would imply
# Last 3 bits of A are 000 and 101 simultaneously
# In general, if B < 3, there is a chance of contradiction

# If B = 0, C must be 0
# If B = 1 (001) then last two bits of C must be 0, so 0 or 4
# if B = 2 (010) then last bit of C must be 0, i.e. even so 0, 2, 4, 6
# If B >= 3 then no restrictions on C as their bits do not overlap

In [144]:
bc_map = defaultdict(set)

for b_ in range(8):
    for c_ in range(8):
        if b_ == 0 and c_ != 0:
            continue
        elif b_ == 1 and c_ % 4 != 0:
            continue
        elif b_ == 2 and c_ % 2 != 0:
            continue
        
        out, *_ = execute_program(0, b_, c_, [1, 2, 1, 3, 4, 4, 5, 5])

        bc_map[out].add((b_, c_))

In [145]:
def determine_bits(program):
    
    nodes = deque([0, [-1]*len(program)*3])

    while nodes:

defaultdict(set,
            {1: {(0, 0), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7)},
             0: {(1, 0), (3, 2), (4, 5), (5, 4), (6, 7), (7, 6)},
             4: {(1, 4), (3, 6), (4, 1), (5, 0), (6, 3), (7, 2)},
             3: {(2, 0), (3, 1), (4, 6), (5, 7), (6, 4), (7, 5)},
             7: {(2, 4), (3, 5), (4, 2), (5, 3), (6, 0), (7, 1)},
             5: {(2, 6), (3, 7), (4, 0), (5, 1), (6, 2), (7, 3)},
             2: {(3, 0), (4, 7), (5, 6), (6, 5), (7, 4)},
             6: {(3, 4), (4, 3), (5, 2), (6, 1), (7, 0)}})

In [148]:
[-1]*len(program)*3

[-1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1]